# **IMPORTAÇÃO DAS BIBLIOTECAS UTILIZADAS E CONEXÃO COM O DRIVE**
PARA EXECUTAR CORRETAMENTE ESTE COLAB:
 - Baixe os arquivos das planilhas e do notebook via github (disponível em: github.com/koitech-API);
 - Em um Google Drive de sua escolha, faça o upload das planilhas e do notebook e execute o código abaixo (pode ser necessário alterar o caminho do repositório no código);
 - Após conceder a permissão de acesso, o código reconhecerá as fontes de dados.

 Obs.: "Executar tudo" fará com que todos os arquivos JSON contendo os gráficos sejam acionados para download.

In [1]:
#BIBLIOTECAS UTILIZADAS
#pandas e numpy (para tratamento dos dados contidos nas planilhas do censo)
import pandas as pd
import numpy as np

#plotly (para construção dos gráficos)
import plotly.express as px
from plotly.utils import PlotlyJSONEncoder
from plotly.subplots import make_subplots
import plotly.graph_objects as go

#google.colab (para acesso e download de arquivos)
from google.colab import drive
from google.colab import files

#json (para converter os gráficos baixados em arquivos .json)
import json

#ACESSANDO O GOOGLE DRIVE PARA TER ACESSO ÀS PLANILHAS
#conectando ao drive
drive.mount('/content/drive')

#listando os arquivos que serão utilizados
!ls "/content/drive/MyDrive/censo_2010_2022"

Mounted at /content/drive
censo_2010_2022.ipynb		 populacao_idade_2010.xlsx
distribuicao_salarial_2010.xlsx  populacao_idade_2022.xlsx
distribuicao_salarial_2022.xlsx  populacao_residente_2010.xlsx
jovens_empregados_2010.xlsx	 populacao_residente_2022.xlsx
jovens_empregados_2022.xlsx	 superior_ciencia_computacao_2010.xlsx
nivel_instrucao_2010.xlsx	 superior_ciencia_computacao_2022.xlsx
nivel_instrucao_2022.xlsx


# **1. CRIAÇÃO E PADRONIZAÇÃO DOS DATAFRAMES**

Os dataframes contém as informações das tabelas selecionadas tratadas para que possam ser utilizadas em seguida do processamento, onde serão feitas os cálculos, em suma, referentes as razões populacionais.


### **1.1 POPULAÇÃO RESIDENTE**
Tabelas do censo demográfico utilizadas: 1378 (2010) e 9514 (2022)

Dataframes: `df_populacao_residente_2010` e `df_populacao_residente_2022`

In [2]:
# 2010
# atribuindo à variável o conteúdo da tabela e pulando as primeiras linhas que podem ser desconsideradas
df_populacao_residente_2010 = pd.read_excel('/content/drive/MyDrive/censo_2010_2022/populacao_residente_2010.xlsx', skiprows=5)

# removendo a coluna da tabela que pode ser desconsiderada e atribuindo as legendas ao cabeçalho do dataframe
df_populacao_residente_2010 = df_populacao_residente_2010.drop(df_populacao_residente_2010.columns[2], axis=1).rename(columns={'Unnamed: 0': 'Municipio', 'Unnamed: 1': 'Idade'})

# atribuindo valores às linhas sem a identificação de munícipio
df_populacao_residente_2010['Municipio'] = df_populacao_residente_2010['Municipio'].ffill()

# utilizando o metódo notna() para remover as linhas sem valores úteis para a análise
df_populacao_residente_2010 = df_populacao_residente_2010[df_populacao_residente_2010['Total'].notna()]

# utilizando o metodo pivot() e reset_index() para "transformar" o dataframe para o formato "longo"
df_populacao_residente_2010 = (df_populacao_residente_2010.pivot(index='Municipio', columns='Idade', values='Total').reset_index())

# removendo o "título" idade da primeira coluna do novo dataframe
df_populacao_residente_2010.columns.name = None

# organizando a coluna "total" para que ela seja exibida após a coluna "municipio"
cols = (['Municipio', 'Total'] + [c for c in df_populacao_residente_2010.columns if c not in ['Municipio', 'Total']])
df_populacao_residente_2010 = df_populacao_residente_2010[cols]

# somando as faixas etárias selecionadas para calcular a população jovem
df_populacao_residente_2010['Jovens'] = df_populacao_residente_2010[['14 anos', '15 a 17 anos', '18 ou 19 anos', '20 a 24 anos', '25 a 29 anos']].sum(axis=1)

# removendo as colunas somadas que não serão mais necessárias no dataframe novo
df_populacao_residente_2010.drop(['14 anos', '15 a 17 anos', '18 ou 19 anos', '20 a 24 anos', '25 a 29 anos'], axis=1, inplace=True)



# 2022
# atribuindo à variável o conteúdo da tabela e pulando as primeiras linhas que podem ser desconsideradas
df_populacao_residente_2022 = pd.read_excel('/content/drive/MyDrive/censo_2010_2022/populacao_residente_2022.xlsx', skiprows=5)

# removendo a coluna da tabela que pode ser desconsiderada e atribuindo as legendas ao cabeçalho do dataframe
df_populacao_residente_2022 = df_populacao_residente_2022.drop(df_populacao_residente_2022.columns[1], axis=1).rename(columns={'Unnamed: 0': 'Municipio', 'Unnamed: 1': 'Idade'})

# utilizando o metódo notna() para remover as linhas sem valores úteis para a análise
df_populacao_residente_2022 = df_populacao_residente_2022[df_populacao_residente_2022['Total'].notna()]

# somando as faixas etárias selecionadas para calcular a população jovem
df_populacao_residente_2022['Jovens'] = df_populacao_residente_2022[['14 anos', '15 a 19 anos', '20 a 24 anos', '25 a 29 anos']].sum(axis=1)

# removendo as colunas somadas que não serão mais necessárias no dataframe novo
df_populacao_residente_2022.drop(['14 anos', '15 a 19 anos', '20 a 24 anos', '25 a 29 anos'], axis=1, inplace=True)

# dispondo o dataframe em ordem alfabética
df_populacao_residente_2022 = df_populacao_residente_2022.sort_values(by="Municipio", ascending=True).reset_index(drop=True)

### **1.2 POPULAÇÃO EMPREGADA**

Tabelas do censo demográfico utilizadas: 3579 (2010) e 10253 (2022)

Dataframes: `df_jovens_empregados_2010` e `df_jovens_empregados_2022`

In [3]:
# 2010
# atribuindo à variável o conteúdo da tabela e pulando as primeiras linhas que podem ser desconsideradas
df_jovens_empregados_2010 = pd.read_excel('/content/drive/MyDrive/censo_2010_2022/jovens_empregados_2010.xlsx', skiprows=6)

# removendo a coluna da tabela que pode ser desconsiderada e atribuindo as legendas ao cabeçalho do dataframe
df_jovens_empregados_2010 = df_jovens_empregados_2010.drop(df_jovens_empregados_2010.columns[1], axis=1).rename(columns={'Unnamed: 0': 'Municipio', 'Unnamed: 2': 'Idade', 'Total.1': 'Empregadas'})

# atribuindo valores às linhas sem a identificação de munícipio
df_jovens_empregados_2010['Municipio'] = df_jovens_empregados_2010['Municipio'].ffill()

# utilizando o metódo notna() para remover as linhas sem valores úteis para a análise
df_jovens_empregados_2010 = df_jovens_empregados_2010[df_jovens_empregados_2010['Total'].notna()]

# utilizando o metodo pivot() e reset_index() para "transformar" o dataframe para o formato "longo"
df_jovens_empregados_2010 = (df_jovens_empregados_2010.pivot_table(index="Municipio", columns="Idade", values=["Total", "Empregadas"]).reset_index())

# remove o nome do eixo de colunas
df_jovens_empregados_2010.columns.name = None

# organizando as colunas por conta das duas métricas utilizadas (empregadas e total)
df_jovens_empregados_2010.columns = [f"{col2} ({col1})" if col2 else col1 for col1, col2 in df_jovens_empregados_2010.columns]

# organizando as colunas "total" para que elas sejam exibidas após a coluna "municipio" com o método pop()
df_jovens_empregados_2010.insert(1, 'Total (Empregadas)', df_jovens_empregados_2010.pop('Total (Empregadas)'))
df_jovens_empregados_2010.insert(6, 'Total (Total)', df_jovens_empregados_2010.pop('Total (Total)'))

# filtrando somente a parcela empregada
df_jovens_empregados_2010 = df_jovens_empregados_2010.drop(df_jovens_empregados_2010.columns[6: 11], axis=1)

# somando as faixas etárias selecionadas para calcular a população jovem
df_jovens_empregados_2010['Jovens (Empregados)'] = df_jovens_empregados_2010[['14 anos (Empregadas)', '15 a 19 anos (Empregadas)', '20 a 24 anos (Empregadas)', '25 a 29 anos (Empregadas)']].sum(axis=1)

# removendo as colunas somadas que não serão mais necessárias no dataframe novo
df_jovens_empregados_2010.drop(['14 anos (Empregadas)', '15 a 19 anos (Empregadas)', '20 a 24 anos (Empregadas)', '25 a 29 anos (Empregadas)'], axis=1, inplace=True)

# renomeando o título da coluna
df_jovens_empregados_2010 = df_jovens_empregados_2010.rename(columns={'Total (Empregadas)': 'População (Empregada)'})



# 2022
# atribuindo à variável o conteúdo da tabela e pulando as primeiras linhas que podem ser desconsideradas
df_jovens_empregados_2022 = pd.read_excel('/content/drive/MyDrive/censo_2010_2022/jovens_empregados_2022.xlsx', skiprows=5)

# removendo a coluna da tabela que pode ser desconsiderada e atribuindo as legendas ao cabeçalho do dataframe
df_jovens_empregados_2022 = df_jovens_empregados_2022.drop(df_jovens_empregados_2022.columns[1], axis=1).rename(columns={'Unnamed: 0': 'Municipio', 'Unnamed: 2': 'Idade', 'Total.1': 'Empregadas'})

# atribuindo valores às linhas sem a identificação de munícipio
df_jovens_empregados_2022['Municipio'] = df_jovens_empregados_2022['Municipio'].ffill()

# utilizando o metódo notna() para remover as linhas sem valores úteis para a análise
df_jovens_empregados_2022 = df_jovens_empregados_2022[df_jovens_empregados_2022['Total'].notna()]

# utilizando o metodo pivot() e reset_index() para "transformar" o dataframe para o formato "longo"
df_jovens_empregados_2022 = (df_jovens_empregados_2022.pivot_table(index="Municipio", columns="Idade", values=["Total", "Empregadas"]).reset_index())

# remove o nome do eixo de colunas
df_jovens_empregados_2022.columns.name = None

# organizando as colunas por conta das duas métricas utilizadas (empregadas e total)
df_jovens_empregados_2022.columns = [f"{col2} ({col1})" if col2 else col1 for col1, col2 in df_jovens_empregados_2022.columns]

# organizando as colunas "total" para que elas sejam exibidas após a coluna "municipio" com o método pop()
df_jovens_empregados_2022.insert(1, 'Total (Empregadas)', df_jovens_empregados_2022.pop('Total (Empregadas)'))
df_jovens_empregados_2022.insert(5, 'Total (Total)', df_jovens_empregados_2022.pop('Total (Total)'))

# filtrando somente a parcela empregada
df_jovens_empregados_2022 = df_jovens_empregados_2022.drop(df_jovens_empregados_2022.columns[5: 9], axis=1)

# somando as faixas etárias selecionadas para calcular a população jovem
df_jovens_empregados_2022['Jovens (Empregados)'] = df_jovens_empregados_2022[['14 a 17 anos (Empregadas)', '18 a 24 anos (Empregadas)', '25 a 29 anos (Empregadas)']].sum(axis=1)

# removendo as colunas somadas que não serão mais necessárias no dataframe novo
df_jovens_empregados_2022.drop(['14 a 17 anos (Empregadas)', '18 a 24 anos (Empregadas)', '25 a 29 anos (Empregadas)'], axis=1, inplace=True)

# renomeando o título da coluna
df_jovens_empregados_2022 = df_jovens_empregados_2022.rename(columns={'Total (Empregadas)': 'População (Empregada)'})

### **1.3 DISTRIBUIÇÃO SALARIAL**
Tabelas do censo demográfico utilizadas: 3569 (2010) e 10381 (2022)

Dataframes: `df_distribuicao_salarial_2010` e `df_distribuicao_salarial_2022`

In [4]:
# 2010
# atribuindo à variável o conteúdo da tabela e pulando as primeiras linhas que podem ser desconsideradas
df_distribuicao_salarial_2010 = pd.read_excel('/content/drive/MyDrive/censo_2010_2022/distribuicao_salarial_2010.xlsx', skiprows=4)

# atribuindo as legendas ao cabeçalho do dataframe
df_distribuicao_salarial_2010 = df_distribuicao_salarial_2010.rename(columns={'Unnamed: 0': 'Municipio', 'Unnamed: 1': 'Faixa Salarial'})

# atribuindo valores às linhas sem a identificação de munícipio
df_distribuicao_salarial_2010['Municipio'] = df_distribuicao_salarial_2010['Municipio'].ffill()

# utilizando o metódo notna() para remover as linhas sem valores úteis para a análise
df_distribuicao_salarial_2010 = df_distribuicao_salarial_2010[df_distribuicao_salarial_2010['Faixa Salarial'].notna()]

# utilizando o metodo pivot() e reset_index() para "transformar" o dataframe para o formato "longo"
df_distribuicao_salarial_2010 = (df_distribuicao_salarial_2010.pivot_table(index="Municipio", columns="Faixa Salarial", values="Economicamente ativa").reset_index())

# remove o nome do eixo de colunas
df_distribuicao_salarial_2010.columns.name = None

# organizando a coluna para que fique em ordem crescente e não em ordem alfabética, por conta do método reset_index()
# lista com a ordem correta
ordem_distribuicao_salarial_2010 = [
    'Total',
    'Até 1/2 salário mínimo',
    'Mais de 1/2 a 1 salário mínimo',
    'Mais de 1 a 2 salários mínimos',
    'Mais de 2 a 5 salários mínimos',
    'Mais de 5 a 10 salários mínimos',
    'Mais de 10 a 20 salários mínimos',
    'Mais de 20 salários mínimos',
    'Sem rendimento'
]

# dispondo as colunas na ordem após a coluna "municipio"
for i, col in enumerate(ordem_distribuicao_salarial_2010, start=1):
    df_distribuicao_salarial_2010.insert(i, col, df_distribuicao_salarial_2010.pop(col))



# 2022
# atribuindo à variável o conteúdo da tabela e pulando as primeiras linhas que podem ser desconsideradas
df_distribuicao_salarial_2022 = pd.read_excel('/content/drive/MyDrive/censo_2010_2022/distribuicao_salarial_2022.xlsx', skiprows=4)

# atribuindo as legendas ao cabeçalho do dataframe
df_distribuicao_salarial_2022 = df_distribuicao_salarial_2022.rename(columns={'Unnamed: 0': 'Municipio', 'Unnamed: 1': 'Faixa Salarial', 'Total': 'Economicamente ativa'})

# tratando os valores não informados no dataframe com o método mask()
df_distribuicao_salarial_2022 = df_distribuicao_salarial_2022.mask(df_distribuicao_salarial_2022 == "-", 0)

# atribuindo valores às linhas sem a identificação de munícipio
df_distribuicao_salarial_2022['Municipio'] = df_distribuicao_salarial_2022['Municipio'].ffill()

# utilizando o metódo notna() para remover as linhas sem valores úteis para a análise
df_distribuicao_salarial_2022 = df_distribuicao_salarial_2022[df_distribuicao_salarial_2022['Faixa Salarial'].notna()]

# utilizando o metodo pivot() e reset_index() para "transformar" o dataframe para o formato "longo"
df_distribuicao_salarial_2022 = (df_distribuicao_salarial_2022.pivot_table(index="Municipio", columns="Faixa Salarial", values="Economicamente ativa").reset_index())

# remove o nome do eixo de colunas
df_distribuicao_salarial_2022.columns.name = None

# organizando a coluna para que fique em ordem crescente e não em ordem alfabética, por conta do método reset_index()
# lista com a ordem correta
ordem_distribuicao_salarial_2022 = [
    'Total',
    'Até 1/4 de salário mínimo',
    'Mais de 1/4 a 1/2 salário mínimo',
    'Mais de 1/2 a 1 salário mínimo',
    'Mais de 1 a 2 salários mínimos',
    'Mais de 2 a 3 salários mínimos',
    'Mais de 3 a 5 salários mínimos',
    'Mais de 5 a 10 salários mínimos',
    'Mais de 10 a 15 salários mínimos',
    'Mais de 15 a 20 salários mínimos',
    'Mais de 20 salários mínimos',
    'Sem rendimento'
]

# dispondo as colunas na ordem após a coluna "municipio"
for i, col in enumerate(ordem_distribuicao_salarial_2022, start=1):
    df_distribuicao_salarial_2022.insert(i, col, df_distribuicao_salarial_2022.pop(col))

# somando
df_distribuicao_salarial_2022['Até 1/2 salário minímo'] = df_distribuicao_salarial_2022[['Até 1/4 de salário mínimo', 'Mais de 1/4 a 1/2 salário mínimo']].sum(axis=1)
df_distribuicao_salarial_2022['Mais de 2 a 5 salários mínimos'] = df_distribuicao_salarial_2022[['Mais de 2 a 3 salários mínimos', 'Mais de 3 a 5 salários mínimos']].sum(axis=1)
df_distribuicao_salarial_2022['Mais de 10 a 20 salários mínimos'] = df_distribuicao_salarial_2022[['Mais de 10 a 15 salários mínimos', 'Mais de 15 a 20 salários mínimos']].sum(axis=1)

# removendo as colunas somadas que não serão mais necessárias no dataframe novo
df_distribuicao_salarial_2022.drop([
    'Até 1/4 de salário mínimo',
    'Mais de 1/4 a 1/2 salário mínimo',
    'Mais de 2 a 3 salários mínimos',
    'Mais de 3 a 5 salários mínimos',
    'Mais de 10 a 15 salários mínimos',
    'Mais de 15 a 20 salários mínimos'
    ], axis=1, inplace=True)

# movendo as colunas de lugar no dataframe
df_distribuicao_salarial_2022.insert(2, 'Até 1/2 salário minímo', df_distribuicao_salarial_2022.pop('Até 1/2 salário minímo'))
df_distribuicao_salarial_2022.insert(5, 'Mais de 2 a 5 salários mínimos', df_distribuicao_salarial_2022.pop('Mais de 2 a 5 salários mínimos'))
df_distribuicao_salarial_2022.insert(7, 'Mais de 10 a 20 salários mínimos', df_distribuicao_salarial_2022.pop('Mais de 10 a 20 salários mínimos'))

### **1.4 NÍVEL DE INSTRUÇÃO**
Tabelas do censo demográfico utilizadas: 3540 (2010) e 10061 (2022)

Dataframes: `df_nivel_instrucao_2010` e `df_nivel_instrucao_2022`

In [5]:
# função para somar as linhas das faixas etárias "jovens"
def soma_nivel_instrucao(df, municipio):
  # seleciona somente as linhas do municipio atual
  municipio_atual = df.loc[df["Municipio"] == municipio]

  # declarando as variaveis para o loop
  total = 0
  sem_instrucao = 0
  fundamental_completo_medio_incompleto = 0
  medio_completo_superior_incompleto = 0
  superior_completo = 0

  # criando o loop responsável por somar as linhas, exceto a linha "total"
  for i in range(1, len(municipio_atual)):
    total += municipio_atual.iloc[i, 2]
    sem_instrucao += municipio_atual.iloc[i, 3]
    fundamental_completo_medio_incompleto += municipio_atual.iloc[i, 4]
    medio_completo_superior_incompleto += municipio_atual.iloc[i, 5]
    superior_completo += municipio_atual.iloc[i, 6]

  # retornando os registros para o dataframe novo
  return {
      "Municipio": municipio,
      "Idade": "Jovens",
      "Total": total,
      "Sem instrução": sem_instrucao,
      "Fundamental completo e médio incompleto": fundamental_completo_medio_incompleto,
      "Médio completo e superior incompleto": medio_completo_superior_incompleto,
      "Superior completo": superior_completo
  }

# selecionando todas as cidades e o estado de sp do dataframe original escolhidas para a análise
municipios = df_populacao_residente_2010['Municipio'].tolist()



# 2010 (por este dataframe conter 5 métricas, os níveis de intrução, não será feita a padronização utilizada nos dataframes anteriores a ele)
# atribuindo à variável o conteúdo da tabela e pulando as primeiras linhas que podem ser desconsideradas
df_nivel_instrucao_2010 = pd.read_excel('/content/drive/MyDrive/censo_2010_2022/nivel_instrucao_2010.xlsx', skiprows=6)

# removendo a coluna que não será necessária e atribuindo as legendas ao cabeçalho do dataframe
df_nivel_instrucao_2010 = df_nivel_instrucao_2010.drop(df_nivel_instrucao_2010.columns[1], axis=1).rename(
    columns={
    'Unnamed: 0': 'Municipio',
    'Unnamed: 2': 'Idade',
    'Total.1': "Sem instrução",
    "Total.2": "Fundamental completo e médio incompleto",
    "Total.3": "Médio completo e superior incompleto",
    "Total.4": "Superior completo"})

# atribuindo valores às linhas sem a identificação de munícipio
df_nivel_instrucao_2010['Municipio'] = df_nivel_instrucao_2010['Municipio'].ffill()

# utilizando o metódo notna() para remover as linhas sem valores úteis para a análise
df_nivel_instrucao_2010 = df_nivel_instrucao_2010[df_nivel_instrucao_2010['Total'].notna()]

# criando uma cópia para filtrar somente a faixa "jovem"
df_nivel_instrucao_jovem_2010 = df_nivel_instrucao_2010.copy()

# tratando os valores não informados no dataframe com o método mask()
df_nivel_instrucao_jovem_2010 = df_nivel_instrucao_jovem_2010.mask(df_nivel_instrucao_jovem_2010 == "-", 0)

# preparando o dataframe "jovem" com a função
resultados_nivel_instrucao_jovem_2010 = []
for m in municipios:
    r = soma_nivel_instrucao(df_nivel_instrucao_jovem_2010, m)
    resultados_nivel_instrucao_jovem_2010.append(r)

# alterando o dataframe "jovem" com os novos registros já filtrados
df_nivel_instrucao_jovem_2010 = pd.DataFrame(resultados_nivel_instrucao_jovem_2010)

# concatenando somente as linhas de interesse para popular o dataframe original
df_nivel_instrucao_2010 = pd.concat([
    df_nivel_instrucao_2010[df_nivel_instrucao_2010["Idade"] == "Total"],
    df_nivel_instrucao_jovem_2010[df_nivel_instrucao_jovem_2010["Idade"] == "Jovens"]],
    ignore_index=True)

# agrupando os registros por município no dataframe original filtrado
df_nivel_instrucao_2010 = df_nivel_instrucao_2010.sort_values(by="Municipio").reset_index(drop=True)



# 2022 (já este dataframe será padronizado para seguir o modelo anterior, o de 2010)
# atribuindo à variável o conteúdo da tabela e pulando as primeiras linhas que podem ser desconsideradas
df_nivel_instrucao_2022 = pd.read_excel("/content/drive/MyDrive/censo_2010_2022/nivel_instrucao_2022.xlsx", header=None)

# encontrando e atribuindo à uma variável o início do conteúdo principal da tabela (linha que contém São Paulo)
inicio = df_nivel_instrucao_2022[df_nivel_instrucao_2022[0].astype(str).str.contains("São Paulo", na=False)].index[0] - 1

# lendo novamente o dataframe com somente os registros relevantes
df_nivel_instrucao_2022 = pd.read_excel("/content/drive/MyDrive/censo_2010_2022/nivel_instrucao_2022.xlsx", skiprows=inicio)

# renomeando o nome das colunas para algo compreensível
df_nivel_instrucao_2022.columns = [
    "Municipio", "Sexo", "Cor_ou_raca",
    "Total", "18 a 24 anos", "25 a 29 anos",
    "Sem instrução e fundamental incompleto",
    "Sem instr. 18-24", "Sem instr. 25-29",
    "Fundamental completo e médio incompleto",
    "Fundam. 18-24", "Fundam. 25-29",
    "Médio completo e superior incompleto",
    "Médio 18-24", "Médio 25-29",
    "Superior completo",
    "Superior 18-24", "Superior 25-29"
]

linhas = []

# criando um loop para percorrer cada linha do dataframe original
for _, row in df_nivel_instrucao_2022.iterrows():
    # criando um segundo loop, pois cada municipio pussui três métricas
    for faixa in ["Total", "18 a 24 anos", "25 a 29 anos"]:
        # seleciona as colunas correspondentes para cada faixa etária
        if faixa == "Total":
            sem_instrucao = row["Sem instrução e fundamental incompleto"]
            fundamental = row["Fundamental completo e médio incompleto"]
            medio = row["Médio completo e superior incompleto"]
            superior = row["Superior completo"]
        elif faixa == "18 a 24 anos":
            sem_instrucao = row["Sem instr. 18-24"]
            fundamental = row["Fundam. 18-24"]
            medio = row["Médio 18-24"]
            superior = row["Superior 18-24"]
        elif faixa == "25 a 29 anos":
            sem_instrucao = row["Sem instr. 25-29"]
            fundamental = row["Fundam. 25-29"]
            medio = row["Médio 25-29"]
            superior = row["Superior 25-29"]

        linhas.append({
            "Municipio": row["Municipio"],
            "Idade": faixa,
            "Total": row[faixa],
            "Sem instrução": sem_instrucao,
            "Fundamental completo e médio incompleto": fundamental,
            "Médio completo e superior incompleto": medio,
            "Superior completo": superior
        })

# alterando o dataframe com a nova formatação
df_nivel_instrucao_2022 = pd.DataFrame(linhas)

# utilizando o metódo notna() para remover as linhas sem valores úteis para a análise
df_nivel_instrucao_2022 = df_nivel_instrucao_2022[df_nivel_instrucao_2022['Total'].notna()]

# criando uma cópia para filtrar somente a faixa "jovem"
df_nivel_instrucao_jovem_2022 = df_nivel_instrucao_2022.copy()

# tratando os valores não informados no dataframe com o método mask()
df_nivel_instrucao_jovem_2022 = df_nivel_instrucao_jovem_2022.mask(df_nivel_instrucao_jovem_2022 == "-", 0)

# preparando o dataframe "jovem" com a função
resultados_nivel_instrucao_jovem_2022 = []
for m in municipios:
    r = soma_nivel_instrucao(df_nivel_instrucao_jovem_2022, m)
    resultados_nivel_instrucao_jovem_2022.append(r)

# alterando o dataframe "jovem" com os novos registros já filtrados
df_nivel_instrucao_jovem_2022 = pd.DataFrame(resultados_nivel_instrucao_jovem_2022)

# concatenando somente as linhas de interesse para popular o dataframe original
df_nivel_instrucao_2022 = pd.concat([
    df_nivel_instrucao_2022[df_nivel_instrucao_2022["Idade"] == "Total"],
    df_nivel_instrucao_jovem_2022[df_nivel_instrucao_jovem_2022["Idade"] == "Jovens"]],
    ignore_index=True)

# agrupando os registros por município no dataframe original filtrado
df_nivel_instrucao_2022 = df_nivel_instrucao_2022.sort_values(by="Municipio").reset_index(drop=True)

### **1.5 SUPERIOR COMPLETO EM CIÊNCIA DA COMPUTAÇÃO**
Tabelas do censo demográfico utilizadas: 3543 (2010) e 10064 (2022)

Dataframes: `df_superior_ciencia_computacao_2010` e `df_superior_ciencia_computacao_2022`

In [6]:
# 2010
# atribuindo à variável o conteúdo da tabela e pulando as primeiras linhas que podem ser desconsideradas
df_superior_ciencia_computacao_2010 = pd.read_excel('/content/drive/MyDrive/censo_2010_2022/superior_ciencia_computacao_2010.xlsx', skiprows=5)

# atribuindo as legendas ao cabeçalho do dataframe
df_superior_ciencia_computacao_2010 = df_superior_ciencia_computacao_2010.rename(columns={'Unnamed: 0': 'Municipio', 'Unnamed: 1': 'Áreas gerais'})

# tratando os valores não informados no dataframe com o método mask()
df_superior_ciencia_computacao_2010 = df_superior_ciencia_computacao_2010.mask(df_superior_ciencia_computacao_2010 == "-", 0)

# atribuindo valores às linhas sem a identificação de munícipio
df_superior_ciencia_computacao_2010['Municipio'] = df_superior_ciencia_computacao_2010['Municipio'].ffill()

# utilizando o metódo notna() para remover as linhas sem valores úteis para a análise
df_superior_ciencia_computacao_2010 = df_superior_ciencia_computacao_2010[df_superior_ciencia_computacao_2010['Total'].notna()]

# utilizando o metodo pivot() e reset_index() para "transformar" o dataframe para o formato "longo"
df_superior_ciencia_computacao_2010 = (df_superior_ciencia_computacao_2010.pivot(index='Municipio', columns='Áreas gerais', values='Total').reset_index())

# renomeando os novos textos dos títulos no cabeçalho
df_superior_ciencia_computacao_2010 = df_superior_ciencia_computacao_2010.rename(columns={"Ciências, matemática e computação - computação - ciência da computação": "Ciência da computação"})

# removendo o "título" idade da primeira coluna do novo dataframe
df_superior_ciencia_computacao_2010.columns.name = None



# 2022
# atribuindo à variável o conteúdo da tabela e pulando as primeiras linhas que podem ser desconsideradas
df_superior_ciencia_computacao_2022 = pd.read_excel('/content/drive/MyDrive/censo_2010_2022/superior_ciencia_computacao_2022.xlsx', skiprows=4)

# atribuindo as legendas ao cabeçalho do dataframe
df_superior_ciencia_computacao_2022 = df_superior_ciencia_computacao_2022.drop(df_superior_ciencia_computacao_2022.columns[1], axis=1).rename(columns={"Unnamed: 0": "Municipio", "0614 - Ciência da computação": "Ciência da computação"})

# utilizando o metódo notna() para remover as linhas sem valores úteis para a análise
df_superior_ciencia_computacao_2022 = df_superior_ciencia_computacao_2022[df_superior_ciencia_computacao_2022['Total'].notna()]

# movendo a coluna de lugar
df_superior_ciencia_computacao_2022.insert(1, 'Ciência da computação', df_superior_ciencia_computacao_2022.pop('Ciência da computação'))

# dispondo o dataframe em ordem alfabética
df_superior_ciencia_computacao_2022 = df_superior_ciencia_computacao_2022.sort_values(by="Municipio", ascending=True).reset_index(drop=True)

### **1.6 DISTRIBUIÇÃO ETÁRIA**
Tabelas do censo demográfico utilizadas: 1552 (2010) e 9514 (2022)

Dataframes: `df_distribuicao_etaria_2010` e `df_distribuicao_etaria_2022`

In [7]:
# 2010
# atribuindo à variável o conteúdo da tabela e pulando as primeiras linhas que podem ser desconsideradas
df_distribuicao_etaria_2010 = pd.read_excel('/content/drive/MyDrive/censo_2010_2022/populacao_idade_2010.xlsx', skiprows=5)

# removendo a coluna da tabela que pode ser desconsiderada e atribuindo as legendas ao cabeçalho do dataframe
df_distribuicao_etaria_2010 = df_distribuicao_etaria_2010.drop(df_distribuicao_etaria_2010.columns[1], axis=1).rename(columns={'Unnamed: 0': 'Municipio', 'Unnamed: 2': 'Idade'})

# atribuindo valores às linhas sem a identificação de munícipio
df_distribuicao_etaria_2010['Municipio'] = df_distribuicao_etaria_2010['Municipio'].ffill()

# utilizando o metódo notna() para remover as linhas sem valores úteis para a análise
df_distribuicao_etaria_2010 = df_distribuicao_etaria_2010[df_distribuicao_etaria_2010['Total'].notna()]

# tratando os valores não informados no dataframe com o método mask()
df_distribuicao_etaria_2010 = df_distribuicao_etaria_2010.mask(df_distribuicao_etaria_2010 == "-", 0)

# utilizando o metodo pivot() e reset_index() para "transformar" o dataframe para o formato "longo"
df_distribuicao_etaria_2010 = (df_distribuicao_etaria_2010.pivot(index='Municipio', columns='Idade', values='Total').reset_index())

# removendo o "título" idade da primeira coluna do novo dataframe
df_distribuicao_etaria_2010.columns.name = None

# organizando a coluna "total" para que ela seja exibida após a coluna "municipio"
cols = (['Municipio', 'Total'] + [c for c in df_distribuicao_etaria_2010.columns if c not in ['Municipio', 'Total']])
df_distribuicao_etaria_2010 = df_distribuicao_etaria_2010[cols]

# movendo as colunas de lugar
tamanho = len(df_distribuicao_etaria_2010.columns)

df_distribuicao_etaria_2010.insert(1, 'Total', df_distribuicao_etaria_2010.pop('Total'))
df_distribuicao_etaria_2010.insert(3, '5 a 9 anos', df_distribuicao_etaria_2010.pop('5 a 9 anos'))
df_distribuicao_etaria_2010.insert(tamanho - 1, '100 anos ou mais', df_distribuicao_etaria_2010.pop('100 anos ou mais'))

# dispondo o dataframe em ordem alfabética
df_distribuicao_etaria_2010 = df_distribuicao_etaria_2010.sort_values(by="Municipio", ascending=True).reset_index(drop=True)



# 2022
# atribuindo à variável o conteúdo da tabela e pulando as primeiras linhas que podem ser desconsideradas
df_distribuicao_etaria_2022 = pd.read_excel('/content/drive/MyDrive/censo_2010_2022/populacao_idade_2022.xlsx', skiprows=5)

# removendo a coluna da tabela que pode ser desconsiderada e atribuindo as legendas ao cabeçalho do dataframe
df_distribuicao_etaria_2022 = df_distribuicao_etaria_2022.drop(df_distribuicao_etaria_2022.columns[1], axis=1).rename(columns={'Unnamed: 0': 'Municipio', 'Unnamed: 2': 'Idade'})

# utilizando o metódo notna() para remover as linhas sem valores úteis para a análise
df_distribuicao_etaria_2022 = df_distribuicao_etaria_2022[df_distribuicao_etaria_2022['Total'].notna()]

# tratando os valores não informados no dataframe com o método mask()
df_distribuicao_etaria_2022 = df_distribuicao_etaria_2022.mask(df_distribuicao_etaria_2022 == "-", 0)

# dispondo o dataframe em ordem alfabética
df_distribuicao_etaria_2022 = df_distribuicao_etaria_2022.sort_values(by="Municipio", ascending=True).reset_index(drop=True)

# **2. PROCESSAMENTO DOS DATAFRAMES DAS TABELAS PARA OS GRÁFICOS**

Seção de análise dos dados dos dataframes criados a partir das tabelas.

### **2.1 PROPORÇÃO DE JOVENS**

Dataframes: `df_razoes_proporcao_jovem_2010` e `df_razoes_proporcao_jovem_2022`

In [8]:
# - recursos que serão utilizados para ambos dataframes -
# criando uma função para retornar as razões de cada cidade
def calcular_razoes_proporcao_jovens(df, municipio, sp_jovem, sp):
  # atribuindo um index para o loop
  idx = df.index[df['Municipio'] == municipio]

  # encontrando o registro que será utilizado
  i = idx[0]

  # calculando as razões
  razao_jovens_populacao_municipio = (df.iloc[i, 2] / df.iloc[i, 1]) * 100
  razao_jovens_jovens_estado = (df.iloc[i, 2] / sp_jovem) * 100
  razao_jovens_populacao_estado = (df.iloc[i, 2] / sp) * 100

  return {
      "Municipio": municipio,
      "Jovens (Município) x População (Município)": f'{razao_jovens_populacao_municipio:.2f}%',
      "Jovens (Município) x Jovens (Estado)": f'{razao_jovens_jovens_estado:.2f}%',
      "Jovens (Município) x População (Estado)": f'{razao_jovens_populacao_estado:.2f}%'
  }

# selecionando todas as cidades do dataframe original escolhidas para a análise, exceto o estado de SP
municipios = df_populacao_residente_2010.loc[df_populacao_residente_2010['Municipio'] != 'São Paulo', 'Municipio'].tolist()



# 2010
# criando uma cópia do dataframe original
df_proporcao_jovens_2010 = df_populacao_residente_2010.copy()

# preparando um dataframe utilizando um loop e a função calcular_razoes passando por cada municipio
populacao_jovem_sp_2010 = df_populacao_residente_2010.loc[df_populacao_residente_2010['Municipio'] == 'São Paulo', 'Jovens'].values[0]
populacao_total_sp_2010 = df_populacao_residente_2010.loc[df_populacao_residente_2010['Municipio'] == 'São Paulo', 'Total'].values[0]

resultados_2010 = []
for m in municipios:
    r = calcular_razoes_proporcao_jovens(df_proporcao_jovens_2010, m, populacao_jovem_sp_2010, populacao_total_sp_2010)
    resultados_2010.append(r)

# alterando o dataframe com as razões de cada cidade
df_razoes_proporcao_jovem_2010 = pd.DataFrame(resultados_2010)



# 2022
# criando uma cópia do dataframe original
df_proporcao_jovens_2022 = df_populacao_residente_2022.copy()

# preparando um dataframe utilizando um loop e a função calcular_razoes passando por cada municipio
populacao_jovem_sp_2022 = df_populacao_residente_2022.loc[df_populacao_residente_2022['Municipio'] == 'São Paulo', 'Jovens'].values[0]
populacao_total_sp_2022 = df_populacao_residente_2022.loc[df_populacao_residente_2022['Municipio'] == 'São Paulo', 'Total'].values[0]

resultados_2022 = []
for m in municipios:
    r = calcular_razoes_proporcao_jovens(df_proporcao_jovens_2022, m, populacao_jovem_sp_2022, populacao_total_sp_2022)
    resultados_2022.append(r)

# alterando o dataframe com as razões de cada cidade
df_razoes_proporcao_jovem_2022 = pd.DataFrame(resultados_2022)

### **2.2 JOVENS EMPREGADOS**

Dataframes: `df_razoes_jovens_empregados_2010` e `df_razoes_jovens_empregados_2022`

In [9]:
# - recursos que serão utilizados para ambos dataframes -
# criando uma função para retornar as razões de cada cidade
def calcular_razoes_emprego(df, municipio, sp_jovem, sp):
  # atribuindo um index para o loop
  idx = df.index[df['Municipio'] == municipio]

  # encontrando o registro que será utilizado
  i = idx[0]

  # calculando as razões
  razao_jovens_populacao_municipio = (df.iloc[i, 2] / df.iloc[i, 1]) * 100
  razao_jovens_jovens_estado = (df.iloc[i, 2] / sp_jovem) * 100
  razao_jovens_populacao_estado = (df.iloc[i, 2] / sp) * 100

  return {
      "Municipio": municipio,
      "Jovens (Município) x População (Município)": f'{razao_jovens_populacao_municipio:.2f}%',
      "Jovens (Município) x Jovens (Estado)": f'{razao_jovens_jovens_estado:.2f}%',
      "Jovens (Município) x População (Estado)": f'{razao_jovens_populacao_estado:.2f}%'
  }

# selecionando todas as cidades do dataframe original escolhidas para a análise, exceto o estado de SP
municipios = df_populacao_residente_2010.loc[df_populacao_residente_2010['Municipio'] != 'São Paulo', 'Municipio'].tolist()



# 2010
# criando uma cópia do dataframe original
df_razoes_jovens_empregados_2010 = df_jovens_empregados_2010.copy()

# preparando um dataframe utilizando um loop e a função calcular_razoes passando por cada municipio
populacao_empregada_sp_2010 = df_razoes_jovens_empregados_2010.loc[df_razoes_jovens_empregados_2010['Municipio'] == 'São Paulo', 'População (Empregada)'].values[0]
populacao_empregada_jovem_sp_2010 = df_razoes_jovens_empregados_2010.loc[df_razoes_jovens_empregados_2010['Municipio'] == 'São Paulo', 'Jovens (Empregados)'].values[0]

resultados_2010 = []
for m in municipios:
    r = calcular_razoes_emprego(df_razoes_jovens_empregados_2010, m, populacao_empregada_jovem_sp_2010, populacao_empregada_sp_2010)
    resultados_2010.append(r)

# alterando o dataframe com as razões de cada cidade
df_razoes_jovens_empregados_2010 = pd.DataFrame(resultados_2010)



# 2022
# criando uma cópia do dataframe original
df_razoes_jovens_empregados_2022 = df_jovens_empregados_2022.copy()

# preparando um dataframe utilizando um loop e a função calcular_razoes passando por cada municipio
populacao_empregada_sp_2022 = df_razoes_jovens_empregados_2022.loc[df_razoes_jovens_empregados_2022['Municipio'] == 'São Paulo', 'População (Empregada)'].values[0]
populacao_empregada_jovem_sp_2022 = df_razoes_jovens_empregados_2022.loc[df_razoes_jovens_empregados_2022['Municipio'] == 'São Paulo', 'Jovens (Empregados)'].values[0]

resultados_2022 = []
for m in municipios:
    r = calcular_razoes_emprego(df_razoes_jovens_empregados_2022, m, populacao_empregada_jovem_sp_2022, populacao_empregada_sp_2022)
    resultados_2022.append(r)

# alterando o dataframe com as razões de cada cidade
df_razoes_jovens_empregados_2022 = pd.DataFrame(resultados_2022)

### **2.3 NÍVEL DE INSTRUÇÃO**

Dataframes: `df_razoes_nivel_instrucao_2010` e `df_razoes_nivel_instrucao_2022`

In [10]:
def calcular_razoes_nivel_instrucao(df, m, c):
  # atribuindo um index para o loop
  idx = df.index[df['Municipio'] == m]

  # encontrando o registro que será utilizado
  i = idx[0]

  # obtendo valores para São Paulo
  coluna = df.columns[c]

  populacao_nivel_instrucao_total_sp = df.loc[(df['Municipio'] == 'São Paulo') & (df['Idade'] == 'Total'), coluna].values[0]
  populacao_nivel_instrucao_jovem_sp = df.loc[(df['Municipio'] == 'São Paulo') & (df['Idade'] == 'Jovens'), coluna].values[0]

  # calculando as razões
  # S.I = Sem instrução | F.C.M.I = Fundamental completo médio incompleto| M.C.S.I = Médio completo superior incompleto | S.C = Superior completo
  # J.M = Jovem Município | P.M = População Município | J.E = Jovem Estado | P.E = População Estado

  # jovem município S.I vs população total do município
  jm_pm_t = (df.iloc[i + 1, c] / df.iloc[i, 2]) * 100
  # jovem município S.I vs jovens totais do município
  jm_jm_t = (df.iloc[i + 1, c] / df.iloc[i + 1, 2]) * 100
  # jovem município S.I vs população total do município sem instrução
  jm_pm_si = (df.iloc[i + 1, c] / df.iloc[i, 3]) * 100
  # jovem município S.I vs jovens do estado sem instrução
  jm_je_si = (df.iloc[i + 1, c] / populacao_nivel_instrucao_jovem_sp) * 100
  # jovem município S.I vs população do estado sem instrução
  jm_pe_si = (df.iloc[i + 1, c] / populacao_nivel_instrucao_total_sp) * 100

  return {
      "Municipio": m,
      "Nível de instrução (J.M)": df.columns[c],
      "vs P.M (Total)": f'{jm_pm_t:.2f}%',
      "vs J.M (Total)": f'{jm_jm_t:.2f}%',
      "vs P.M (S.I)": f'{jm_pm_si:.2f}%',
      "vs J.E (S.I)": f'{jm_je_si:.2f}%',
      "vs P.E (S.I)": f'{jm_pe_si:.2f}%'
  }

# selecionando todas as cidades do dataframe original escolhidas para a análise, exceto o estado de SP
municipios = df_populacao_residente_2010.loc[df_populacao_residente_2010['Municipio'] != 'São Paulo', 'Municipio'].tolist()



# 2010
# criando uma cópia do dataframe original
df_razoes_nivel_instrucao_2010  = df_nivel_instrucao_2010.copy()

# preparando um dataframe para cada nível de instrução
resultados_nivel_instrucao_2010 = []
colunas = 3
while colunas <= 6:
  for m in municipios:
    r = calcular_razoes_nivel_instrucao(df_razoes_nivel_instrucao_2010, m, colunas)
    resultados_nivel_instrucao_2010.append(r)
  colunas += 1

# alterando o dataframe original
df_razoes_nivel_instrucao_2010 = pd.DataFrame(resultados_nivel_instrucao_2010)

# agrupando os registros por município no dataframe original filtrado
df_razoes_nivel_instrucao_2010 = df_razoes_nivel_instrucao_2010.sort_values(by="Municipio").reset_index(drop=True)



# 2022
# criando uma cópia do dataframe original
df_razoes_nivel_instrucao_2022  = df_nivel_instrucao_2022.copy()

# preparando um dataframe para cada nível de instrução
resultados_nivel_instrucao_2022 = []
colunas = 3
while colunas <= 6:
  for m in municipios:
    r = calcular_razoes_nivel_instrucao(df_razoes_nivel_instrucao_2022, m, colunas)
    resultados_nivel_instrucao_2022.append(r)
  colunas += 1

# alterando o dataframe original
df_razoes_nivel_instrucao_2022 = pd.DataFrame(resultados_nivel_instrucao_2022)

# agrupando os registros por município no dataframe original filtrado
df_razoes_nivel_instrucao_2022 = df_razoes_nivel_instrucao_2022.sort_values(by="Municipio").reset_index(drop=True)

### **2.4 SUPERIOR EM CIÊNCIA DA COMPUTAÇÃO**
Dataframes: `df_razoes_superior_ciencia_computacao_2010` e `df_razoes_superior_ciencia_computacao_2022`

In [11]:
# - recursos que serão utilizados para ambos dataframes -
# criando uma função para retornar as razões de cada cidade
def calcular_razoes_superior_ciencia_computacao(df_ciencia_computacao, df_residentes, municipio):
  # atribuindo um index para o loop
  idx = df_ciencia_computacao.index[df_ciencia_computacao['Municipio'] == municipio]
  idx2 = df_residentes.index[df_residentes['Municipio'] == municipio]

  # encontrando o registro que será utilizado
  i = idx[0]
  ii = idx2[0]

  # calculando as razões
  razao_formados_municipio = (df_ciencia_computacao.iloc[i, 2] / df_residentes.iloc[ii, 1]) * 100
  razao_formados_ciencia_computacao_municipio = (df_ciencia_computacao.iloc[i, 1] / df_residentes.iloc[ii, 1]) * 100

  return {
      "Municipio": municipio,
      "Formandos Total x População Município": f'{razao_formados_municipio:.2f}%',
      "Formandos Ciência da Computação x População Município": f'{razao_formados_ciencia_computacao_municipio:.2f}%'
  }

# selecionando todas as cidades do dataframe original escolhidas para a análise, exceto o estado de SP
municipios = df_populacao_residente_2010.loc[df_populacao_residente_2010['Municipio'] != 'São Paulo', 'Municipio'].tolist()



# 2010
# criando uma cópia do dataframe original
df_razoes_superior_ciencia_computacao_2010 = df_superior_ciencia_computacao_2010.copy()

# preparando um dataframe utilizando um loop e a função calcular_razoes passando por cada municipio
resultados_superior_ciencia_computacao_2010 = []
for m in municipios:
    r = calcular_razoes_superior_ciencia_computacao(df_razoes_superior_ciencia_computacao_2010, df_populacao_residente_2010, m)
    resultados_superior_ciencia_computacao_2010.append(r)

# alterando o dataframe com as razões de cada cidade
df_razoes_superior_ciencia_computacao_2010 = pd.DataFrame(resultados_superior_ciencia_computacao_2010)



# 2022
# criando uma cópia do dataframe original
df_razoes_superior_ciencia_computacao_2022 = df_superior_ciencia_computacao_2022.copy()

# preparando um dataframe utilizando um loop e a função calcular_razoes passando por cada municipio
resultados_superior_ciencia_computacao_2022 = []
for m in municipios:
    r = calcular_razoes_superior_ciencia_computacao(df_razoes_superior_ciencia_computacao_2022, df_populacao_residente_2022, m)
    resultados_superior_ciencia_computacao_2022.append(r)

# alterando o dataframe com as razões de cada cidade
df_razoes_superior_ciencia_computacao_2022 = pd.DataFrame(resultados_superior_ciencia_computacao_2022)

### **2.5 DISTRIBUIÇÃO SALARIAL**
Dataframes: `df_razoes_distribuicao_salarial_2010` e `df_razoes_distribuicao_salarial_2022`

In [12]:
def calcular_razoes_distribuicao_salarial(df, m, c):
  # atribuindo um index para o loop
  idx = df.index[df['Municipio'] == m]

  # encontrando o registro que será utilizado
  i = idx[0]

  # obtendo valores para São Paulo
  coluna = df.columns[c]

  # calculando as razões

  # jovem município S.I vs população total do município
  faixa_salarial_municipio = (df.iloc[i, c] / df.iloc[i, 1]) * 100

  return {
      "Municipio": m,
      "Faixa Salarial": df.columns[c],
      "Faixa x População Total Munícipio": f'{faixa_salarial_municipio:.2f}%'
  }

# selecionando todas as cidades do dataframe original escolhidas para a análise, exceto o estado de SP
municipios = df_populacao_residente_2010.loc[df_populacao_residente_2010['Municipio'] != 'São Paulo', 'Municipio'].tolist()



# 2010
# criando uma cópia do dataframe original
df_razoes_distribuicao_salarial_2010 = df_distribuicao_salarial_2010.copy()

# preparando um dataframe para cada nível de instrução
resultados_distribuicao_salarial_2010 = []
colunas = 2
while colunas <= 9:
  for m in municipios:
    r = calcular_razoes_distribuicao_salarial(df_razoes_distribuicao_salarial_2010, m, colunas)
    resultados_distribuicao_salarial_2010.append(r)
  colunas += 1

# alterando o dataframe original
df_razoes_distribuicao_salarial_2010 = pd.DataFrame(resultados_distribuicao_salarial_2010)



# 2022
# criando uma cópia do dataframe original
df_razoes_distribuicao_salarial_2022 = df_distribuicao_salarial_2022.copy()

# preparando um dataframe para cada nível de instrução
resultados_distribuicao_salarial_2022 = []
colunas = 2
while colunas <= 9:
  for m in municipios:
    r = calcular_razoes_distribuicao_salarial(df_razoes_distribuicao_salarial_2022, m, colunas)
    resultados_distribuicao_salarial_2022.append(r)
  colunas += 1

# alterando o dataframe original
df_razoes_distribuicao_salarial_2022 = pd.DataFrame(resultados_distribuicao_salarial_2022)

# **3. CRIAÇÃO DOS GRÁFICOS**

Seção onde os dados obtidos nas análises dos dataframes das tabelas do censo são exibidos gráficamente.

### **3.1 PROPORÇÃO DE JOVENS**

Gráficos: `grafico_proporcao_jovens_2010` e `grafico_proporcao_jovens_2022`

In [13]:
# 2010
# transformando o dataframe para exibição no gráfico
df_razoes_proporcao_jovem_2010_melt = df_razoes_proporcao_jovem_2010.melt(
    id_vars=["Municipio"],
    var_name="Indicador",
    value_name="Percentual"
)

# removendo o "%" dos registros e transformando-os em float para exibição no gráfico
df_razoes_proporcao_jovem_2010_melt["Percentual"] = (
    df_razoes_proporcao_jovem_2010_melt["Percentual"]
    .str.replace("%", "", regex=False)
    .astype(float)
)

# criando a estrutura das barras do gráfico
grafico_proporcao_jovens_2010 = px.bar(
    df_razoes_proporcao_jovem_2010_melt,
    x="Municipio",
    y="Percentual",
    color="Indicador",
    barmode="group",
    text="Percentual"
)

# atribuindo um título estilizado ao gráfico
grafico_proporcao_jovens_2010.update_layout(
    title={
        'text': "Proporção de Jovens (2010)<br><sup>Cidade x Estado | Jovens x Cidade | Jovens x Estado</sup>",
        'x': 0.5, 'xanchor': 'center'
    }
)

# exibindo a porcentagem acima de cada barra
grafico_proporcao_jovens_2010.update_traces(texttemplate='%{text}%', textposition='outside')

# criando o gráfico em si
grafico_proporcao_jovens_2010.update_layout(
    yaxis_title="Percentual (%)",
    xaxis_title="Município",
    uniformtext_minsize=5,
    uniformtext_mode='hide',
    bargap=0.1
)



# 2022
# transformando o dataframe para exibição no gráfico
df_razoes_proporcao_jovem_2022_melt = df_razoes_proporcao_jovem_2022.melt(
    id_vars=["Municipio"],
    var_name="Indicador",
    value_name="Percentual"
)

# removendo o "%" dos registros e transformando-os em float para exibição no gráfico
df_razoes_proporcao_jovem_2022_melt["Percentual"] = (
    df_razoes_proporcao_jovem_2022_melt["Percentual"]
    .str.replace("%", "", regex=False)
    .astype(float)
)

# criando a estrutura das barras do gráfico
grafico_proporcao_jovens_2022 = px.bar(
    df_razoes_proporcao_jovem_2022_melt,
    x="Municipio",
    y="Percentual",
    color="Indicador",
    barmode="group",
    text="Percentual"
)

# atribuindo um título estilizado ao gráfico
grafico_proporcao_jovens_2022.update_layout(
    title={
        'text': "Proporção de Jovens (2022)<br><sup>Cidade x Estado | Jovens x Cidade | Jovens x Estado</sup>",
        'x': 0.5, 'xanchor': 'center'
    }
)

# exibindo a porcentagem acima de cada barra
grafico_proporcao_jovens_2022.update_traces(texttemplate='%{text}%', textposition='outside')

# criando o gráfico em si
grafico_proporcao_jovens_2022.update_layout(
    yaxis_title="Percentual (%)",
    xaxis_title="Município",
    uniformtext_minsize=5,
    uniformtext_mode='hide',
    bargap=0.1
)

print()

### **3.2 JOVENS EMPREGADOS**

Gráficos: `grafico_jovens_empregados_2010` e `grafico_jovens_empregados_2022`

In [14]:
# 2010
# transformando o dataframe para exibição no gráfico
df_razoes_jovens_empregados_2010_melt = df_razoes_jovens_empregados_2010.melt(
    id_vars=["Municipio"],
    var_name="Indicador (População Empregada Total e Jovem)",
    value_name="Percentual"
)

# removendo o "%" dos registros e transformando-os em float para exibição no gráfico
df_razoes_jovens_empregados_2010_melt["Percentual"] = (
    df_razoes_jovens_empregados_2010_melt["Percentual"]
    .str.replace("%", "", regex=False)
    .astype(float)
)

# criando a estrutura das barras do gráfico
grafico_jovens_empregados_2010 = px.bar(
    df_razoes_jovens_empregados_2010_melt,
    x="Municipio",
    y="Percentual",
    color="Indicador (População Empregada Total e Jovem)",
    barmode="group",
    text="Percentual"
)

# atribuindo um título estilizado ao gráfico
grafico_jovens_empregados_2010.update_layout(
    title={
        'text': "Proporção de Jovens Empregados (2010)<br><sup>Jovens (Município) x População (Município)	 | Jovens (Município) x Jovens (Estado) | Jovens (Município) x População (Estado)</sup>",
        'x': 0.4, 'xanchor': 'center'
    }
)

# exibindo a porcentagem acima de cada barra
grafico_jovens_empregados_2010.update_traces(texttemplate='%{text}%', textposition='outside')

# criando o gráfico em si
grafico_jovens_empregados_2010.update_layout(
    yaxis_title="Percentual (%)",
    xaxis_title="Município",
    uniformtext_minsize=5,
    uniformtext_mode='hide',
    bargap=0.1
)



# 2022
# transformando o dataframe para exibição no gráfico
df_razoes_jovens_empregados_2022_melt = df_razoes_jovens_empregados_2022.melt(
    id_vars=["Municipio"],
    var_name="Indicador (População Empregada Total e Jovem)",
    value_name="Percentual"
)

# removendo o "%" dos registros e transformando-os em float para exibição no gráfico
df_razoes_jovens_empregados_2022_melt["Percentual"] = (
    df_razoes_jovens_empregados_2022_melt["Percentual"]
    .str.replace("%", "", regex=False)
    .astype(float)
)

# criando a estrutura das barras do gráfico
grafico_jovens_empregados_2022 = px.bar(
    df_razoes_jovens_empregados_2022_melt,
    x="Municipio",
    y="Percentual",
    color="Indicador (População Empregada Total e Jovem)",
    barmode="group",
    text="Percentual"
)

# atribuindo um título estilizado ao gráfico
grafico_jovens_empregados_2022.update_layout(
    title={
        'text': "Proporção de Jovens Empregados (2022)<br><sup>Jovens (Município) x População (Município)	 | Jovens (Município) x Jovens (Estado) | Jovens (Município) x População (Estado)</sup>",
        'x': 0.4, 'xanchor': 'center'
    }
)

# exibindo a porcentagem acima de cada barra
grafico_jovens_empregados_2022.update_traces(texttemplate='%{text}%', textposition='outside')

# criando o gráfico em si
grafico_jovens_empregados_2022.update_layout(
    yaxis_title="Percentual (%)",
    xaxis_title="Município",
    uniformtext_minsize=5,
    uniformtext_mode='hide',
    bargap=0.1
)

print()

### **3.3 NÍVEL DE INSTRUÇÃO**
Gráficos: `grafico_nivel_instrucao_2010` e `grafico_nivel_instrucao_2022`

In [15]:
# 2010
# transformando o dataframe para exibição no gráfico
df_razoes_nivel_instrucao_2010_melt = df_razoes_nivel_instrucao_2010.melt(
    id_vars=["Municipio", "Nível de instrução (J.M)"],
    value_vars=["vs P.M (Total)", "vs J.M (Total)", "vs P.M (S.I)", "vs J.E (S.I)", "vs P.E (S.I)"],
    var_name="Comparação",
    value_name="Percentual"
)

# removendo o "%" dos registros e transformando-os em float para exibição no gráfico
df_razoes_nivel_instrucao_2010_melt["Percentual"] = (
    df_razoes_nivel_instrucao_2010_melt["Percentual"]
    .str.replace("%", "", regex=False)
    .astype(float)
)

# criando a estrutura das barras do gráfico
grafico_nivel_instrucao_2010 = px.bar(
    df_razoes_nivel_instrucao_2010_melt,
    x="Nível de instrução (J.M)",
    y="Percentual",
    color="Comparação",
    barmode="group",
    facet_col="Municipio",
    facet_row_spacing=0.02,
    facet_col_wrap=1,
    text="Percentual"
)

# atribuindo um título estilizado ao gráfico
# S.I = Sem instrução | F.C.M.I = Fundamental completo médio incompleto| M.C.S.I = Médio completo superior incompleto | S.C = Superior completo
# J.M = Jovem Município | P.M = População Município | J.E = Jovem Estado | P.E = População Estado
grafico_nivel_instrucao_2010.update_layout(
    title={
        'text': "Nível de instrução da população jovem por município (2010)<br><sup>" +
        "S.I = Sem instrução | F.C.M.I = Fundamental completo médio incompleto| M.C.S.I = Médio completo superior incompleto | S.C = Superior completo</sup><br>" +
        "<sup>J.M = Jovem Município | P.M = População Município | J.E = Jovem Estado | P.E = População Estado</sup>",
        'x': 0.45, 'xanchor': 'center',
        'y': 0.99, 'yanchor': 'top'
    }
)

# margem no topo para alocar o título do gráfico
grafico_nivel_instrucao_2010.update_layout(
    margin=dict(t=120)
)

# tamanho dinâmico da altura interna dos gráficos
num_municipios = df_razoes_nivel_instrucao_2010_melt["Municipio"].nunique()

# criando o gráfico em si
grafico_nivel_instrucao_2010.update_layout(
    xaxis_title="",
    yaxis_title="Percentual (%)",
    legend_title="Tipo de comparação",
    height=500 * num_municipios
)

# exibindo a porcentagem acima de cada barra
grafico_nivel_instrucao_2010.update_traces(texttemplate='%{text}%', textposition='outside')

# consertando os eixos do gráfico
grafico_nivel_instrucao_2010.update_yaxes(autorange=True)
grafico_nivel_instrucao_2010.update_xaxes(showticklabels=True)



# 2022
# transformando o dataframe para exibição no gráfico
df_razoes_nivel_instrucao_2022_melt = df_razoes_nivel_instrucao_2022.melt(
    id_vars=["Municipio", "Nível de instrução (J.M)"],
    value_vars=["vs P.M (Total)", "vs J.M (Total)", "vs P.M (S.I)", "vs J.E (S.I)", "vs P.E (S.I)"],
    var_name="Comparação",
    value_name="Percentual"
)

# removendo o "%" dos registros e transformando-os em float para exibição no gráfico
df_razoes_nivel_instrucao_2022_melt["Percentual"] = (
    df_razoes_nivel_instrucao_2022_melt["Percentual"]
    .str.replace("%", "", regex=False)
    .astype(float)
)

# criando a estrutura das barras do gráfico
grafico_nivel_instrucao_2022 = px.bar(
    df_razoes_nivel_instrucao_2022_melt,
    x="Nível de instrução (J.M)",
    y="Percentual",
    color="Comparação",
    barmode="group",
    facet_col="Municipio",
    facet_row_spacing=0.02,
    facet_col_wrap=1,
    text="Percentual"
)

# atribuindo um título estilizado ao gráfico
# S.I = Sem instrução | F.C.M.I = Fundamental completo médio incompleto| M.C.S.I = Médio completo superior incompleto | S.C = Superior completo
# J.M = Jovem Município | P.M = População Município | J.E = Jovem Estado | P.E = População Estado
grafico_nivel_instrucao_2022.update_layout(
    title={
        'text': "Nível de instrução da população jovem por município (2022)<br><sup>" +
        "S.I = Sem instrução | F.C.M.I = Fundamental completo médio incompleto| M.C.S.I = Médio completo superior incompleto | S.C = Superior completo</sup><br>" +
        "<sup>J.M = Jovem Município | P.M = População Município | J.E = Jovem Estado | P.E = População Estado</sup>",
        'x': 0.45, 'xanchor': 'center',
        'y': 0.99, 'yanchor': 'top'
    }
)

# margem no topo para alocar o título do gráfico
grafico_nivel_instrucao_2022.update_layout(
    margin=dict(t=120)
)

# tamanho dinâmico da altura interna dos gráficos
num_municipios = df_razoes_nivel_instrucao_2022_melt["Municipio"].nunique()

# criando o gráfico em si
grafico_nivel_instrucao_2022.update_layout(
    xaxis_title="",
    yaxis_title="Percentual (%)",
    legend_title="Tipo de comparação",
    height=500 * num_municipios
)

# exibindo a porcentagem acima de cada barra
grafico_nivel_instrucao_2022.update_traces(texttemplate='%{text}%', textposition='outside')

# consertando os eixos do gráfico
grafico_nivel_instrucao_2022.update_yaxes(autorange=True)

# faixa salarial em baixo de cada gráfico gerado
grafico_nivel_instrucao_2022.update_xaxes(showticklabels=True)

print()

### **3.4 DISTRIBUIÇÃO ETÁRIA**
Gráficos: grafico_distribuicao_etaria_2010 e grafico_distribuicao_etaria_2022

In [16]:
# 2010
# friltrando os municípios com exeção do estado de SP
df_distribuicao_etaria_2010 = df_distribuicao_etaria_2010[df_distribuicao_etaria_2010['Municipio'] != 'São Paulo']

# transformando o dataframe para o formato "longo"
df_distribuicao_etaria_2010_melt = df_distribuicao_etaria_2010.melt(id_vars=['Municipio', 'Total'],
                  var_name='Faixa Etária',
                  value_name='População')

# mantendo a ordem original das colunas etárias
faixas = [col for col in df_distribuicao_etaria_2010.columns if col not in ['Municipio', 'Total']]
df_distribuicao_etaria_2010_melt['Faixa Etária'] = pd.Categorical(df_distribuicao_etaria_2010_melt['Faixa Etária'],
                                         categories=faixas,
                                         ordered=True)

# criando um gráfico para cada município
grafico_distribuicao_etaria_2010 = px.bar(df_distribuicao_etaria_2010_melt,
             x='População',
             y='Faixa Etária',
             orientation='h',
             # linha responsável por cada município:
             facet_col='Municipio',
             facet_col_wrap=1,
             facet_row_spacing=0.02,
             title='Distribuição Etária - Municípios (2010)',
             text='População')

# ajustes
grafico_distribuicao_etaria_2010.update_traces(textposition='outside')
grafico_distribuicao_etaria_2010.update_layout(
    xaxis_title='População',
    yaxis_title='Faixa Etária',
    bargap=0.1,
    height=3000,  # altura total da figura
    showlegend=False
)

# atribuindo o título ao gráfico
grafico_distribuicao_etaria_2010.update_layout(
    title={
        'text': "Distribuição etária por município (2010)",
        'x': 0.5, 'xanchor': 'center'
    }
)



# 2022
# friltrando os municípios com exeção do estado de SP
df_distribuicao_etaria_2022 = df_distribuicao_etaria_2022[df_distribuicao_etaria_2022['Municipio'] != 'São Paulo']

# transformando o dataframe para o formato "longo"
df_distribuicao_etaria_2022_melt = df_distribuicao_etaria_2022.melt(id_vars=['Municipio', 'Total'],
                  var_name='Faixa Etária',
                  value_name='População')

# mantendo a ordem original das colunas etárias
faixas = [col for col in df_distribuicao_etaria_2022.columns if col not in ['Municipio', 'Total']]
df_distribuicao_etaria_2022_melt['Faixa Etária'] = pd.Categorical(df_distribuicao_etaria_2022_melt['Faixa Etária'],
                                         categories=faixas,
                                         ordered=True)

# criando um gráfico para cada município
grafico_distribuicao_etaria_2022 = px.bar(df_distribuicao_etaria_2022_melt,
             x='População',
             y='Faixa Etária',
             orientation='h',
             # linha responsável por cada município:
             facet_col='Municipio',
             facet_col_wrap=1,
             facet_row_spacing=0.02,
             title='Distribuição Etária - Municípios (2010)',
             text='População')

# ajustes
grafico_distribuicao_etaria_2022.update_traces(textposition='outside')
grafico_distribuicao_etaria_2022.update_layout(
    xaxis_title='População',
    yaxis_title='Faixa Etária',
    bargap=0.1,
    height=3000,  # altura total da figura
    showlegend=False
)

# atribuindo o título ao gráfico
grafico_distribuicao_etaria_2022.update_layout(
    title={
        'text': "Distribuição etária por município (2022)",
        'x': 0.5, 'xanchor': 'center'
    }
)

print()

### **3.5 SUPERIOR EM CIÊNCIA DA COMPUTAÇÃO**
Gráficos: `grafico_superior_ciencia_computacao_2010` e `grafico_superior_ciencia_computacao_2022`

In [17]:
# 2010
# transformando o dataframe para exibição no gráfico
df_razoes_superior_ciencia_computacao_2010_melt = df_razoes_superior_ciencia_computacao_2010.melt(
    id_vars=["Municipio"],
    var_name="Indicador",
    value_name="Percentual"
)

# removendo o "%" dos registros e transformando-os em float para exibição no gráfico
df_razoes_superior_ciencia_computacao_2010_melt["Percentual"] = (
    df_razoes_superior_ciencia_computacao_2010_melt["Percentual"]
    .str.replace("%", "", regex=False)
    .astype(float)
)

# criando a estrutura das barras do gráfico
grafico_superior_ciencia_computacao_2010 = px.bar(
    df_razoes_superior_ciencia_computacao_2010_melt,
    x="Municipio",
    y="Percentual",
    color="Indicador",
    barmode="group",
    text="Percentual"
)

# atribuindo um título estilizado ao gráfico
grafico_superior_ciencia_computacao_2010.update_layout(
    title={
        'text': "Proporção de Pessoas Formadas em Ciência da Computação - 2010",
        'x': 0.5, 'xanchor': 'center'
    }
)

# exibindo a porcentagem acima de cada barra
grafico_superior_ciencia_computacao_2010.update_traces(texttemplate='%{text}%', textposition='outside')

# criando o gráfico em si
grafico_superior_ciencia_computacao_2010.update_layout(
    yaxis_title="Percentual (%)",
    xaxis_title="Município",
    uniformtext_minsize=5,
    uniformtext_mode='hide',
    bargap=0.1
)



# 2022
# transformando o dataframe para exibição no gráfico
df_razoes_superior_ciencia_computacao_2022_melt = df_razoes_superior_ciencia_computacao_2022.melt(
    id_vars=["Municipio"],
    var_name="Indicador",
    value_name="Percentual"
)

# removendo o "%" dos registros e transformando-os em float para exibição no gráfico
df_razoes_superior_ciencia_computacao_2022_melt["Percentual"] = (
    df_razoes_superior_ciencia_computacao_2022_melt["Percentual"]
    .str.replace("%", "", regex=False)
    .astype(float)
)

# criando a estrutura das barras do gráfico
grafico_superior_ciencia_computacao_2022 = px.bar(
    df_razoes_superior_ciencia_computacao_2022_melt,
    x="Municipio",
    y="Percentual",
    color="Indicador",
    barmode="group",
    text="Percentual"
)

# atribuindo um título estilizado ao gráfico
grafico_superior_ciencia_computacao_2022.update_layout(
    title={
        'text': "Proporção de Pessoas Formadas em Ciência da Computação - 2022",
        'x': 0.5, 'xanchor': 'center'
    }
)

# exibindo a porcentagem acima de cada barra
grafico_superior_ciencia_computacao_2022.update_traces(texttemplate='%{text}%', textposition='outside')

# criando o gráfico em si
grafico_superior_ciencia_computacao_2022.update_layout(
    yaxis_title="Percentual (%)",
    xaxis_title="Município",
    uniformtext_minsize=5,
    uniformtext_mode='hide',
    bargap=0.1
)

print()

### **3.6 DISTRIBUIÇÃO SALARIAL**
Dataframes: `grafico_distribuicao_salarial_2010` e `grafico_distribuicao_salarial_2022`

In [18]:
# 2010
# transformando o dataframe para exibição no gráfico
df_razoes_distribuicao_salarial_2010_melt = df_razoes_distribuicao_salarial_2010.melt(
    id_vars=["Municipio", "Faixa Salarial"],
    value_vars=["Faixa x População Total Munícipio"],
    value_name="Percentual"
)

# removendo o "%" dos registros e transformando-os em float para exibição no gráfico
df_razoes_distribuicao_salarial_2010_melt["Percentual"] = (
    df_razoes_distribuicao_salarial_2010_melt["Percentual"]
    .str.replace("%", "", regex=False)
    .astype(float)
)

# criando a estrutura das barras do gráfico
grafico_distribuicao_salarial_2010 = px.bar(
    df_razoes_distribuicao_salarial_2010_melt,
    x="Faixa Salarial",
    y="Percentual",
    barmode="group",
    facet_col="Municipio",
    facet_row_spacing=0.05,
    facet_col_wrap=1,
    text="Percentual"
)

# atribuindo um título estilizado ao gráfico
grafico_distribuicao_salarial_2010.update_layout(
    title={
        'text': "Distribuição Salarial por Munícipio - 2010",
        'x': 0.5, 'xanchor': 'center'
    }
)

# tamanho dinâmico da altura interna dos gráficos
num_municipios = df_razoes_distribuicao_salarial_2010_melt["Municipio"].nunique()

# criando o gráfico em si
grafico_distribuicao_salarial_2010.update_layout(
    xaxis_title="",
    yaxis_title="Percentual (%)",
    legend_title="Tipo de comparação",
    height= 800 * num_municipios
)

# exibindo a porcentagem acima de cada barra
grafico_distribuicao_salarial_2010.update_traces(texttemplate='%{text}%', textposition='outside')

# faixa salarial em baixo de cada gráfico gerado
grafico_distribuicao_salarial_2010.update_xaxes(showticklabels=True)



# 2022
# transformando o dataframe para exibição no gráfico
df_razoes_distribuicao_salarial_2022_melt = df_razoes_distribuicao_salarial_2022.melt(
    id_vars=["Municipio", "Faixa Salarial"],
    value_vars=["Faixa x População Total Munícipio"],
    value_name="Percentual"
)

# removendo o "%" dos registros e transformando-os em float para exibição no gráfico
df_razoes_distribuicao_salarial_2022_melt["Percentual"] = (
    df_razoes_distribuicao_salarial_2022_melt["Percentual"]
    .str.replace("%", "", regex=False)
    .astype(float)
)

# criando a estrutura das barras do gráfico
grafico_distribuicao_salarial_2022 = px.bar(
    df_razoes_distribuicao_salarial_2022_melt,
    x="Faixa Salarial",
    y="Percentual",
    barmode="group",
    facet_col="Municipio",
    facet_row_spacing=0.05,
    facet_col_wrap=1,
    text="Percentual"
)

# atribuindo um título estilizado ao gráfico
grafico_distribuicao_salarial_2022.update_layout(
    title={
        'text': "Distribuição Salarial por Munícipio - 2022",
        'x': 0.5, 'xanchor': 'center'
    }
)

# tamanho dinâmico da altura interna dos gráficos
num_municipios = df_razoes_distribuicao_salarial_2022_melt["Municipio"].nunique()

# criando o gráfico em si
grafico_distribuicao_salarial_2022.update_layout(
    xaxis_title="",
    yaxis_title="Percentual (%)",
    legend_title="Tipo de comparação",
    height= 800 * num_municipios
)

# exibindo a porcentagem acima de cada barra
grafico_distribuicao_salarial_2022.update_traces(texttemplate='%{text}%', textposition='outside')

# faixa salarial em baixo de cada gráfico gerado
grafico_distribuicao_salarial_2022.update_xaxes(showticklabels=True)

print()

# **EXIBIÇÃO - DATAFRAMES - TABELAS**

População Residente

In [ ]:
df_populacao_residente_2010

,Municipio,Total,Jovens
0,Caçapava (SP),84752.0,23265.0
1,Jacareí (SP),211214.0,56479.0
2,Joanópolis (SP),11768.0,3151.0
3,Sapucaí-Mirim (MG),6241.0,1651.0
4,São José dos Campos (SP),629921.0,178996.0
5,São Paulo,41262199.0,11402597.0
6,São Paulo (SP),11253503.0,3081060.0


In [ ]:
df_populacao_residente_2022

,Municipio,Total,Jovens
0,Caçapava (SP),96202.0,20425.0
1,Jacareí (SP),240275.0,51907.0
2,Joanópolis (SP),12815.0,2613.0
3,Sapucaí-Mirim (MG),6311.0,1419.0
4,São José dos Campos (SP),697054.0,153442.0
5,São Paulo,44411238.0,9918311.0
6,São Paulo (SP),11451999.0,2582698.0


Jovens Empregados

In [ ]:
df_jovens_empregados_2010

,Municipio,População (Empregada),Jovens (Empregados)
0,Caçapava (SP),37264.0,12266.0
1,Jacareí (SP),97247.0,31160.0
2,Joanópolis (SP),6124.0,2020.0
3,Sapucaí-Mirim (MG),2888.0,1035.0
4,São José dos Campos (SP),311297.0,103877.0
5,São Paulo,20001270.0,6528679.0
6,São Paulo (SP),5549787.0,1752536.0


In [ ]:
df_jovens_empregados_2022

,Municipio,População (Empregada),Jovens (Empregados)
0,Caçapava (SP),43062.0,10210.0
1,Jacareí (SP),112640.0,27993.0
2,Joanópolis (SP),6680.0,1622.0
3,Sapucaí-Mirim (MG),3073.0,876.0
4,São José dos Campos (SP),332022.0,81241.0
5,São Paulo,21327751.0,5297448.0
6,São Paulo (SP),5596991.0,1360296.0


Distribuição Salarial

In [ ]:
df_distribuicao_salarial_2010

,Municipio,Total,Até 1/2 salário mínimo,Mais de 1/2 a 1 salário mínimo,Mais de 1 a 2 salários mínimos,Mais de 2 a 5 salários mínimos,Mais de 5 a 10 salários mínimos,Mais de 10 a 20 salários mínimos,Mais de 20 salários mínimos,Sem rendimento
0,Caçapava (SP),40754.0,1424.0,6032.0,14515.0,10164.0,3856.0,1005.0,435.0,3322.0
1,Jacareí (SP),107818.0,3678.0,15975.0,39313.0,27221.0,8464.0,2537.0,650.0,9980.0
2,Joanópolis (SP),6376.0,375.0,1900.0,2421.0,1112.0,243.0,44.0,18.0,264.0
3,Sapucaí-Mirim (MG),3037.0,248.0,1063.0,1149.0,310.0,70.0,22.0,12.0,162.0
4,São José dos Campos (SP),336282.0,7960.0,42093.0,114890.0,88385.0,35568.0,15135.0,7550.0,24702.0
5,São Paulo,21639776.0,617590.0,2810343.0,8126808.0,5534951.0,1867403.0,699150.0,357243.0,1626288.0
6,São Paulo (SP),5998740.0,107687.0,643767.0,2060648.0,1541569.0,682401.0,317595.0,193908.0,451165.0


In [ ]:
df_distribuicao_salarial_2022

,Municipio,Total,Até 1/2 salário minímo,Mais de 1/2 a 1 salário mínimo,Mais de 1 a 2 salários mínimos,Mais de 2 a 5 salários mínimos,Mais de 5 a 10 salários mínimos,Mais de 10 a 20 salários mínimos,Mais de 20 salários mínimos,Sem rendimento
0,Caçapava (SP),43062.0,3484.0,7581.0,17705.0,11200.0,1982.0,648.0,185.0,277.0
1,Jacareí (SP),112640.0,6420.0,19013.0,43823.0,33226.0,7009.0,2126.0,474.0,550.0
2,Joanópolis (SP),6680.0,441.0,1657.0,2681.0,1447.0,284.0,70.0,35.0,65.0
3,Sapucaí-Mirim (MG),3073.0,239.0,1026.0,1245.0,468.0,61.0,19.0,5.0,10.0
4,São José dos Campos (SP),332022.0,16862.0,49716.0,120450.0,98799.0,27707.0,11486.0,4044.0,2959.0
5,São Paulo,21327751.0,1029823.0,3315370.0,8171158.0,6514179.0,1410033.0,555043.0,210800.0,121344.0
6,São Paulo (SP),5596991.0,243923.0,840280.0,1858975.0,1709107.0,530703.0,261824.0,115559.0,36621.0


Nível de Instrução

In [ ]:
df_nivel_instrucao_2010

,Municipio,Idade,Total,Sem instrução,Fundamental completo e médio incompleto,Médio completo e superior incompleto,Superior completo
0,Caçapava (SP),Total,72988.0,28633.0,13369.0,23104.0,7119
1,Caçapava (SP),Jovens,17687.0,2547.0,3226.0,9871.0,1728
2,Jacareí (SP),Total,181807.0,71359.0,34421.0,58228.0,17012
3,Jacareí (SP),Jovens,42750.0,5875.0,8834.0,24327.0,3463
4,Joanópolis (SP),Total,10216.0,6012.0,1859.0,1643.0,632
5,Joanópolis (SP),Jovens,2400.0,783.0,595.0,846.0,134
6,Sapucaí-Mirim (MG),Total,5282.0,3456.0,830.0,753.0,226
7,Sapucaí-Mirim (MG),Jovens,1166.0,430.0,292.0,385.0,52
8,São José dos Campos (SP),Total,543951.0,192120.0,102812.0,171177.0,74261
9,São José dos Campos (SP),Jovens,137467.0,17680.0,27569.0,73388.0,17288


In [ ]:
df_nivel_instrucao_2022

,Municipio,Idade,Total,Sem instrução,Fundamental completo e médio incompleto,Médio completo e superior incompleto,Superior completo
0,Caçapava (SP),Total,74273.0,17585.0,10841.0,33125.0,12722.0
1,Caçapava (SP),Jovens,15459.0,1391.0,2181.0,9938.0,1948.0
2,Jacareí (SP),Total,186335.0,40652.0,22540.0,83892.0,39252.0
3,Jacareí (SP),Jovens,39269.0,2108.0,5146.0,26262.0,5751.0
4,Joanópolis (SP),Total,9973.0,4109.0,1534.0,2856.0,1474.0
5,Joanópolis (SP),Jovens,1922.0,327.0,425.0,898.0,272.0
6,Sapucaí-Mirim (MG),Total,4851.0,2081.0,700.0,1619.0,452.0
7,Sapucaí-Mirim (MG),Jovens,1060.0,104.0,202.0,669.0,85.0
8,São José dos Campos (SP),Total,541849.0,96985.0,74037.0,228774.0,142053.0
9,São José dos Campos (SP),Jovens,118964.0,5422.0,18529.0,73153.0,21859.0


Superior em Ciência da Computação

In [ ]:
df_superior_ciencia_computacao_2010

,Municipio,Ciência da computação,Total
0,Caçapava (SP),210,7119
1,Jacareí (SP),440,17012
2,Joanópolis (SP),0,632
3,Sapucaí-Mirim (MG),0,226
4,São José dos Campos (SP),1851,74261
5,São Paulo,100671,4171221
6,São Paulo (SP),44855,1572070


In [ ]:
df_superior_ciencia_computacao_2022

,Municipio,Ciência da computação,Total
0,Caçapava (SP),144.0,12722.0
1,Jacareí (SP),449.0,39252.0
2,Joanópolis (SP),14.0,1474.0
3,Sapucaí-Mirim (MG),13.0,452.0
4,São José dos Campos (SP),2473.0,142053.0
5,São Paulo,103132.0,7478587.0
6,São Paulo (SP),38759.0,2512570.0


Distribuição Etária

In [ ]:
df_distribuicao_etaria_2010

,Municipio,Total,0 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,...,45 a 49 anos,50 a 54 anos,55 a 59 anos,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 a 89 anos,90 a 99 anos,100 anos ou mais
0,Caçapava (SP),84752.0,5710.0,5909.0,6938.0,6961.0,7240.0,7663.0,7188.0,6561.0,...,5676.0,5037.0,4121.0,3285.0,2355.0,1662.0,1176.0,1131.0,148.0,6.0
1,Jacareí (SP),211214.0,14162.0,15096.0,17561.0,17069.0,17507.0,18318.0,18105.0,16626.0,...,14415.0,12695.0,10675.0,8186.0,5623.0,4078.0,2633.0,2379.0,355.0,19.0
2,Joanópolis (SP),11768.0,736.0,783.0,823.0,993.0,1010.0,986.0,876.0,864.0,...,795.0,741.0,647.0,517.0,399.0,316.0,234.0,208.0,31.0,2.0
3,Sapucaí-Mirim (MG),6241.0,435.0,515.0,637.0,536.0,471.0,514.0,512.0,418.0,...,394.0,364.0,279.0,233.0,188.0,139.0,90.0,82.0,13.0,2.0
4,São José dos Campos (SP),629921.0,42187.0,43702.0,51275.0,52022.0,56351.0,60282.0,56240.0,48632.0,...,42772.0,38154.0,31163.0,22688.0,14741.0,10115.0,6752.0,6632.0,1016.0,42.0
6,São Paulo (SP),11253503.0,710927.0,758279.0,867430.0,842257.0,991659.0,1074583.0,1010076.0,888684.0,...,742720.0,667658.0,548113.0,423055.0,302338.0,237301.0,170969.0,176716.0,26732.0,1027.0


In [ ]:
df_distribuicao_etaria_2022

,Municipio,Total,0 a 4 anos,5 a 9 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,...,55 a 59 anos,60 a 64 anos,65 a 69 anos,70 a 74 anos,75 a 79 anos,80 a 84 anos,85 a 89 anos,90 a 94 anos,95 a 99 anos,100 anos ou mais
0,Caçapava (SP),96202.0,5756.0,6178.0,6220.0,6114.0,6585.0,6535.0,7018.0,8127.0,...,5881.0,5269.0,4196.0,3046.0,1982.0,1246.0,628.0,261.0,59.0,13
1,Jacareí (SP),240275.0,13300.0,15621.0,15597.0,15631.0,16400.0,16874.0,17981.0,19805.0,...,14668.0,12681.0,10355.0,7835.0,4950.0,2812.0,1428.0,606.0,135.0,17
2,Joanópolis (SP),12815.0,716.0,793.0,827.0,850.0,755.0,823.0,945.0,1000.0,...,808.0,775.0,605.0,509.0,323.0,214.0,96.0,48.0,13.0,0
3,Sapucaí-Mirim (MG),6311.0,378.0,400.0,416.0,433.0,453.0,440.0,437.0,491.0,...,414.0,361.0,296.0,227.0,141.0,80.0,36.0,18.0,3.0,0
4,São José dos Campos (SP),697054.0,39987.0,44960.0,43908.0,44369.0,49806.0,50555.0,53937.0,57604.0,...,41337.0,36989.0,30242.0,22173.0,13818.0,7851.0,4044.0,1800.0,491.0,85
6,São Paulo (SP),11451999.0,599931.0,682444.0,670931.0,720787.0,845139.0,882865.0,888568.0,942674.0,...,682446.0,606419.0,490960.0,370447.0,244904.0,161775.0,92602.0,42666.0,11526.0,1761


# **EXIBIÇÃO - DATAFRAMES - ANÁLISES**

Proporção de Jovens

In [ ]:
df_razoes_proporcao_jovem_2010

,Municipio,Jovens (Município) x População (Município),Jovens (Município) x Jovens (Estado),Jovens (Município) x População (Estado)
0,Caçapava (SP),27.45%,0.20%,0.06%
1,Jacareí (SP),26.74%,0.50%,0.14%
2,Joanópolis (SP),26.78%,0.03%,0.01%
3,Sapucaí-Mirim (MG),26.45%,0.01%,0.00%
4,São José dos Campos (SP),28.42%,1.57%,0.43%
5,São Paulo (SP),27.38%,27.02%,7.47%


In [ ]:
df_razoes_proporcao_jovem_2022

,Municipio,Jovens (Município) x População (Município),Jovens (Município) x Jovens (Estado),Jovens (Município) x População (Estado)
0,Caçapava (SP),21.23%,0.21%,0.05%
1,Jacareí (SP),21.60%,0.52%,0.12%
2,Joanópolis (SP),20.39%,0.03%,0.01%
3,Sapucaí-Mirim (MG),22.48%,0.01%,0.00%
4,São José dos Campos (SP),22.01%,1.55%,0.35%
5,São Paulo (SP),22.55%,26.04%,5.82%


Jovens Empregados

In [ ]:
df_razoes_jovens_empregados_2010

,Municipio,Jovens (Município) x População (Município),Jovens (Município) x Jovens (Estado),Jovens (Município) x População (Estado)
0,Caçapava (SP),32.92%,0.19%,0.06%
1,Jacareí (SP),32.04%,0.48%,0.16%
2,Joanópolis (SP),32.98%,0.03%,0.01%
3,Sapucaí-Mirim (MG),35.84%,0.02%,0.01%
4,São José dos Campos (SP),33.37%,1.59%,0.52%
5,São Paulo (SP),31.58%,26.84%,8.76%


In [ ]:
df_razoes_jovens_empregados_2022

,Municipio,Jovens (Município) x População (Município),Jovens (Município) x Jovens (Estado),Jovens (Município) x População (Estado)
0,Caçapava (SP),23.71%,0.19%,0.05%
1,Jacareí (SP),24.85%,0.53%,0.13%
2,Joanópolis (SP),24.28%,0.03%,0.01%
3,Sapucaí-Mirim (MG),28.51%,0.02%,0.00%
4,São José dos Campos (SP),24.47%,1.53%,0.38%
5,São Paulo (SP),24.30%,25.68%,6.38%


Nível de Instrução

In [ ]:
df_razoes_nivel_instrucao_2010

,Municipio,Nível de instrução (J.M),vs P.M (Total),vs J.M (Total),vs P.M (S.I),vs J.E (S.I),vs P.E (S.I)
0,Caçapava (SP),Sem instrução,3.49%,14.40%,8.90%,0.16%,0.02%
1,Caçapava (SP),Médio completo e superior incompleto,13.52%,55.81%,34.47%,0.23%,0.10%
2,Caçapava (SP),Superior completo,2.37%,9.77%,6.03%,0.17%,0.04%
3,Caçapava (SP),Fundamental completo e médio incompleto,4.42%,18.24%,11.27%,0.17%,0.05%
4,Jacareí (SP),Médio completo e superior incompleto,13.38%,56.91%,34.09%,0.58%,0.25%
5,Jacareí (SP),Fundamental completo e médio incompleto,4.86%,20.66%,12.38%,0.47%,0.13%
6,Jacareí (SP),Superior completo,1.90%,8.10%,4.85%,0.34%,0.08%
7,Jacareí (SP),Sem instrução,3.23%,13.74%,8.23%,0.38%,0.04%
8,Joanópolis (SP),Fundamental completo e médio incompleto,5.82%,24.79%,9.90%,0.03%,0.01%
9,Joanópolis (SP),Sem instrução,7.66%,32.62%,13.02%,0.05%,0.01%


In [ ]:
df_razoes_nivel_instrucao_2022

,Municipio,Nível de instrução (J.M),vs P.M (Total),vs J.M (Total),vs P.M (S.I),vs J.E (S.I),vs P.E (S.I)
0,Caçapava (SP),Sem instrução,1.87%,9.00%,7.91%,0.24%,0.02%
1,Caçapava (SP),Médio completo e superior incompleto,13.38%,64.29%,56.51%,0.22%,0.07%
2,Caçapava (SP),Superior completo,2.62%,12.60%,11.08%,0.15%,0.03%
3,Caçapava (SP),Fundamental completo e médio incompleto,2.94%,14.11%,12.40%,0.16%,0.04%
4,Jacareí (SP),Médio completo e superior incompleto,14.09%,66.88%,64.60%,0.59%,0.19%
5,Jacareí (SP),Fundamental completo e médio incompleto,2.76%,13.10%,12.66%,0.39%,0.10%
6,Jacareí (SP),Superior completo,3.09%,14.65%,14.15%,0.45%,0.08%
7,Jacareí (SP),Sem instrução,1.13%,5.37%,5.19%,0.37%,0.03%
8,Joanópolis (SP),Fundamental completo e médio incompleto,4.26%,22.11%,10.34%,0.03%,0.01%
9,Joanópolis (SP),Sem instrução,3.28%,17.01%,7.96%,0.06%,0.00%


Superior em Ciência da Computação

In [ ]:
df_razoes_superior_ciencia_computacao_2010

,Municipio,Formandos Total x População Município,Formandos Ciência da Computação x População Município
0,Caçapava (SP),8.40%,0.25%
1,Jacareí (SP),8.05%,0.21%
2,Joanópolis (SP),5.37%,0.00%
3,Sapucaí-Mirim (MG),3.62%,0.00%
4,São José dos Campos (SP),11.79%,0.29%
5,São Paulo (SP),13.97%,0.40%


In [ ]:
df_razoes_superior_ciencia_computacao_2022

,Municipio,Formandos Total x População Município,Formandos Ciência da Computação x População Município
0,Caçapava (SP),13.22%,0.15%
1,Jacareí (SP),16.34%,0.19%
2,Joanópolis (SP),11.50%,0.11%
3,Sapucaí-Mirim (MG),7.16%,0.21%
4,São José dos Campos (SP),20.38%,0.35%
5,São Paulo (SP),21.94%,0.34%


Distribuição Salarial

In [ ]:
df_razoes_distribuicao_salarial_2010

,Municipio,Faixa Salarial,Faixa x População Total Munícipio
0,Caçapava (SP),Até 1/2 salário mínimo,3.49%
1,Jacareí (SP),Até 1/2 salário mínimo,3.41%
2,Joanópolis (SP),Até 1/2 salário mínimo,5.88%
3,Sapucaí-Mirim (MG),Até 1/2 salário mínimo,8.17%
4,São José dos Campos (SP),Até 1/2 salário mínimo,2.37%
5,São Paulo (SP),Até 1/2 salário mínimo,1.80%
6,Caçapava (SP),Mais de 1/2 a 1 salário mínimo,14.80%
7,Jacareí (SP),Mais de 1/2 a 1 salário mínimo,14.82%
8,Joanópolis (SP),Mais de 1/2 a 1 salário mínimo,29.80%
9,Sapucaí-Mirim (MG),Mais de 1/2 a 1 salário mínimo,35.00%


In [ ]:
df_razoes_distribuicao_salarial_2022

,Municipio,Faixa Salarial,Faixa x População Total Munícipio
0,Caçapava (SP),Até 1/2 salário minímo,8.09%
1,Jacareí (SP),Até 1/2 salário minímo,5.70%
2,Joanópolis (SP),Até 1/2 salário minímo,6.60%
3,Sapucaí-Mirim (MG),Até 1/2 salário minímo,7.78%
4,São José dos Campos (SP),Até 1/2 salário minímo,5.08%
5,São Paulo (SP),Até 1/2 salário minímo,4.36%
6,Caçapava (SP),Mais de 1/2 a 1 salário mínimo,17.60%
7,Jacareí (SP),Mais de 1/2 a 1 salário mínimo,16.88%
8,Joanópolis (SP),Mais de 1/2 a 1 salário mínimo,24.81%
9,Sapucaí-Mirim (MG),Mais de 1/2 a 1 salário mínimo,33.39%


# **EXIBIÇÃO - GRÁFICOS**

Proporção de Jovens

In [19]:
grafico_proporcao_jovens_2010.show()
grafico_proporcao_jovens_2022.show()

Jovens Empregados

In [20]:
grafico_jovens_empregados_2010.show()
grafico_jovens_empregados_2022.show()

Nível de Instrução

In [21]:
grafico_nivel_instrucao_2010.show()
grafico_nivel_instrucao_2022.show()

Distribuição Etária

In [22]:
grafico_distribuicao_etaria_2010.show()
grafico_distribuicao_etaria_2022.show()

Superior em Ciência da Computação

In [23]:
grafico_superior_ciencia_computacao_2010.show()
grafico_superior_ciencia_computacao_2022.show()

Distribuição Salarial

In [24]:
grafico_distribuicao_salarial_2010.show()
grafico_distribuicao_salarial_2022.show()

# **DOWNLOAD - GRÁFICOS (ARQUIVOS JSON)**

Proporção de Jovens

In [ ]:
#salva o arquivo JSON
grafico_proporcao_jovens_2010 = json.dumps(grafico_proporcao_jovens_2010, cls=PlotlyJSONEncoder)

with open("grafico_proporcao_jovens_2010.json", "w") as f:
    f.write(grafico_proporcao_jovens_2010)

#faz o download
files.download("grafico_proporcao_jovens_2010.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#salva o arquivo JSON
grafico_proporcao_jovens_2022 = json.dumps(grafico_proporcao_jovens_2022, cls=PlotlyJSONEncoder)

with open("grafico_proporcao_jovens_2022.json", "w") as f:
    f.write(grafico_proporcao_jovens_2022)

#faz o download
files.download("grafico_proporcao_jovens_2022.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Jovens Empregados

In [ ]:
#salva o arquivo JSON
grafico_jovens_empregados_2010 = json.dumps(grafico_jovens_empregados_2010, cls=PlotlyJSONEncoder)

with open("grafico_jovens_empregados_2010.json", "w") as f:
    f.write(grafico_jovens_empregados_2010)

#faz o download
files.download("grafico_jovens_empregados_2010.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#salva o arquivo JSON
grafico_jovens_empregados_2022 = json.dumps(grafico_jovens_empregados_2022, cls=PlotlyJSONEncoder)

with open("grafico_jovens_empregados_2022.json", "w") as f:
    f.write(grafico_jovens_empregados_2022)

#faz o download
files.download("grafico_jovens_empregados_2022.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Nível de Instrução

In [ ]:
#salva o arquivo JSON
grafico_nivel_instrucao_2010 = json.dumps(grafico_nivel_instrucao_2010, cls=PlotlyJSONEncoder)

with open("grafico_nivel_instrucao_2010.json", "w") as f:
    f.write(grafico_nivel_instrucao_2010)

#faz o download
files.download("grafico_nivel_instrucao_2010.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#salva o arquivo JSON
grafico_nivel_instrucao_2022 = json.dumps(grafico_nivel_instrucao_2022, cls=PlotlyJSONEncoder)

with open("grafico_nivel_instrucao_2022.json", "w") as f:
    f.write(grafico_nivel_instrucao_2022)

#faz o download
files.download("grafico_nivel_instrucao_2022.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Distribuição Etária

In [ ]:
#salva o arquivo JSON
grafico_distribuicao_etaria_2010 = json.dumps(grafico_distribuicao_etaria_2010, cls=PlotlyJSONEncoder)

with open("grafico_distribuicao_etaria_2010.json", "w") as f:
    f.write(grafico_distribuicao_etaria_2010)

#faz o download
files.download("grafico_distribuicao_etaria_2010.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#salva o arquivo JSON
grafico_distribuicao_etaria_2022 = json.dumps(grafico_distribuicao_etaria_2022, cls=PlotlyJSONEncoder)

with open("grafico_distribuicao_etaria_2022.json", "w") as f:
    f.write(grafico_distribuicao_etaria_2022)

#faz o download
files.download("grafico_distribuicao_etaria_2022.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Superior em Ciência da Computação

In [ ]:
#salva o arquivo JSON
grafico_superior_ciencia_computacao_2010 = json.dumps(grafico_superior_ciencia_computacao_2010, cls=PlotlyJSONEncoder)

with open("grafico_superior_ciencia_computacao_2010.json", "w") as f:
    f.write(grafico_superior_ciencia_computacao_2010)

#faz o download
files.download("grafico_superior_ciencia_computacao_2010.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#salva o arquivo JSON
grafico_superior_ciencia_computacao_2022 = json.dumps(grafico_superior_ciencia_computacao_2022, cls=PlotlyJSONEncoder)

with open("grafico_superior_ciencia_computacao_2022.json", "w") as f:
    f.write(grafico_superior_ciencia_computacao_2022)

#faz o download
files.download("grafico_superior_ciencia_computacao_2022.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Distribuição Salarial

In [ ]:
#salva o arquivo JSON
grafico_distribuicao_salarial_2010 = json.dumps(grafico_distribuicao_salarial_2010, cls=PlotlyJSONEncoder)

with open("grafico_distribuicao_salarial_2010.json", "w") as f:
    f.write(grafico_distribuicao_salarial_2010)

#faz o download
files.download("grafico_distribuicao_salarial_2010.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
#salva o arquivo JSON
grafico_distribuicao_salarial_2022 = json.dumps(grafico_distribuicao_salarial_2022, cls=PlotlyJSONEncoder)

with open("grafico_distribuicao_salarial_2022.json", "w") as f:
    f.write(grafico_distribuicao_salarial_2022)

#faz o download
files.download("grafico_distribuicao_salarial_2022.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>